# Recordati - List
## Beachten
* PDF zuerst entschlüsseln!
* Für die OCR-Erkennung wurde ABBYY Fine Reader for Mac verwendet (Lizenz vorhanden)
* Wichtig: Sprachen einstellen auf: Deutsch, Englisch, Italienisch, Französisch
* Als CSV exportieren
* In Google Sheet bearbeiten
* Wieder importieren
* Set Type **manually**

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import consts

## Unlock PDF

In [2]:
import pikepdf
pdf = pikepdf.open('2018_Offenlegung_Recordati.pdf')
pdf.save('unlocked.pdf')

## Import CSV

In [3]:
# Read CSV into DataFrame
header = ['name', 'location', 'country', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', ]
df = pd.read_csv("recordati tmp - Tabellenblatt1.csv", sep=",", header=None, names=header, dtype = dataframe_types_numbers_as_string)

## Format Table

In [5]:
df_export = df.copy()

#Set Type
index_hco = df_export[df_export['name'].str.contains("Association D'Esculape", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove Carination
df_export = remove_carination(df_export, ' ')

#Clean Numbers
df_export = cleanup_number(df_export)
df_export = remove_in_numbers(df_export, '-')
df_export = replace_in_number(df_export, ',', '.')
df_export = replace_in_number(df_export, 'O', '0')
df_export = replace_in_number(df_export, 'I', '1')
df_export = replace_in_number(df_export, 'T', '1')
df_export = replace_in_number(df_export, '*', '')

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)
df_export = add_uci(df_export)

In [6]:
def add_decimal_if_not(field):
    if field != 'ERROR' and field != '' and field != '0':
        f = str(field)
        return f[:-2] + '.' + f[-2]
    else:
        return field

for field in number_fields:
    df_export[field] = df_export[field].fillna("")
    df_export.loc[df_export[field].str.contains(r'\.', regex=True, na=False) == False, field] = df_export[field].apply(add_decimal_if_not)


In [7]:
#Export
df_export = amounts_to_number(df_export)
add_warning(manually=True)
df_export = amounts_to_number(df_export)
export_list(df_export, 'recordati')

Total nicht Summe der Werte
saved


In [8]:
#write_to_csv(df_export, 'tmp.csv', True)
write_to_excel(df_export, 'tmp.xlsx', open=True)